In [15]:
import pandas as pd
from datetime import date, timedelta

In [36]:
df_cases_daily = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-data.csv")
df_cases_agg = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-table-data.csv")
df_vac = pd.read_csv("https://covid19.who.int/who-data/vaccination-data.csv")
df_vac_meta = pd.read_csv("https://covid19.who.int/who-data/vaccination-metadata.csv")
df_public_health = pd.read_csv("https://covid19.who.int/who-data/phsm-severity-data.csv")

In [37]:
for df in [df_cases_daily, df_cases_agg, df_vac, df_vac_meta, df_public_health]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144570 entries, 0 to 144569
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      144570 non-null  object
 1   Country_code       143960 non-null  object
 2   Country            144570 non-null  object
 3   WHO_region         144570 non-null  object
 4   New_cases          144570 non-null  int64 
 5   Cumulative_cases   144570 non-null  int64 
 6   New_deaths         144570 non-null  int64 
 7   Cumulative_deaths  144570 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 8.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, Global to Tuvalu
Data columns (total 12 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Name                                                          237

In [38]:
today = date.today()
yesterday = today - timedelta(days=1)
daysAgo7 = today - timedelta(days=7)
yesterday_str = yesterday.strftime("%Y-%m-%d")
daysAgo7_str = daysAgo7.strftime("%Y-%m-%d")

In [51]:
df_7daysAgo = df_cases_daily[df_cases_daily["Date_reported"] >= daysAgo7_str]
df_details_agg = df_7daysAgo.groupby(["Country_code", "Country", "WHO_region"]).agg(
    date = pd.NamedAgg(column="Date_reported", aggfunc=max),
    cases7days = pd.NamedAgg(column="New_cases", aggfunc=sum),
    deaths7days = pd.NamedAgg(column="New_deaths", aggfunc=sum)
)
df_details_agg.reset_index(inplace=True)
df_details_agg = df_details_agg[["Country","date","cases7days","deaths7days"]]
df_details_agg

,Country,date,cases7days,deaths7days
0,Other,2021-09-03,0,0
1,Andorra,2021-09-03,27,0
2,United Arab Emirates,2021-09-03,5934,8
3,Afghanistan,2021-09-03,368,24
4,Antigua and Barbuda,2021-09-03,144,1
...,...,...,...,...
231,Yemen,2021-09-03,240,39
232,Mayotte,2021-09-03,136,0
233,South Africa,2021-09-03,49387,1727
234,Zambia,2021-09-03,1001,21


In [48]:
df_vac["pop"] = 100*df_vac["TOTAL_VACCINATIONS"]/df_vac["TOTAL_VACCINATIONS_PER100"]
df_vac_red = df_vac[["COUNTRY", "PERSONS_VACCINATED_1PLUS_DOSE_PER100", "PERSONS_FULLY_VACCINATED_PER100"]]
df_vac_red.rename(columns={"COUNTRY": "Country", "PERSONS_VACCINATED_1PLUS_DOSE_PER100": "Perc1Vacc", "PERSONS_FULLY_VACCINATED_PER100": "PercFullyVacc"}, inplace=True)
df_vac_red

C:\Users\nikol\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Country,Perc1Vacc,PercFullyVacc
0,Afghanistan,1.986,1.134
1,Albania,28.639,22.026
2,Algeria,7.802,1.653
3,American Samoa,54.338,45.251
4,Andorra,66.064,52.567
...,...,...,...
222,Viet Nam,17.062,2.296
223,Wallis and Futuna,44.033,43.118
224,Yemen,1.000,0.045
225,Zambia,1.653,1.086


In [41]:
df_vac

,COUNTRY,ISO3,WHO_REGION,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,PERSONS_VACCINATED_1PLUS_DOSE,TOTAL_VACCINATIONS_PER100,PERSONS_VACCINATED_1PLUS_DOSE_PER100,PERSONS_FULLY_VACCINATED,PERSONS_FULLY_VACCINATED_PER100,VACCINES_USED,FIRST_VACCINE_DATE,NUMBER_VACCINES_TYPES_USED,pop
0,Afghanistan,AFG,EMRO,REPORTING,2021-08-30,1979652,773002.0,5.085,1.986,441371.0,1.134,"Beijing CNBG - BBIBP-CorV,Janssen - Ad26.COV 2...",2021-02-22,4.0,3.893121e+07
1,Albania,ALB,EURO,OWID,2021-08-30,1458048,824170.0,50.665,28.639,633878.0,22.026,"AstraZeneca - AZD1222, Pfizer BioNTech - Comir...",NaN,4.0,2.877821e+06
2,Algeria,DZA,AFRO,REPORTING,2021-08-23,4146091,3421279.0,9.455,7.802,724812.0,1.653,"Beijing CNBG - BBIBP-CorV,Gamaleya - Sputnik V...",2021-01-30,3.0,4.385078e+07
3,American Samoa,ASM,WPRO,REPORTING,2021-08-20,54425,29993.0,98.601,54.338,24977.0,45.251,"Janssen - Ad26.COV 2.5,Moderna - mRNA-1273,Pfi...",2020-12-21,3.0,5.519721e+04
4,Andorra,AND,EURO,OWID,2021-08-15,91660,51044.0,118.631,66.064,40616.0,52.567,"AstraZeneca - AZD1222, Pfizer BioNTech - Comir...",NaN,2.0,7.726480e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Viet Nam,VNM,WPRO,REPORTING,2021-08-27,18843004,16607991.0,19.358,17.062,2235013.0,2.296,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-03-08,5.0,9.733962e+07
223,Wallis and Futuna,WLF,WPRO,REPORTING,2021-07-22,9801,4952.0,87.151,44.033,4849.0,43.118,Moderna - mRNA-1273,2021-03-19,1.0,1.124600e+04
224,Yemen,YEM,EMRO,REPORTING,2021-07-27,311483,298161.0,1.044,1.000,13322.0,0.045,SII - Covishield,2021-04-20,1.0,2.983554e+07
225,Zambia,ZMB,AFRO,REPORTING,2021-08-23,503707,303966.0,2.740,1.653,199741.0,1.086,SII - Covishield,2021-04-14,1.0,1.838347e+07


In [49]:
result = pd.merge(df_details_agg, df_vac_red, how="inner", on=["Country"])

In [50]:
result

,Country,date,cases7days,deaths7days,Perc1Vacc,PercFullyVacc
0,Andorra,2021-09-03,27,0,66.064,52.567
1,United Arab Emirates,2021-09-03,5934,8,NaN,NaN
2,Afghanistan,2021-09-03,368,24,1.986,1.134
3,Antigua and Barbuda,2021-09-03,144,1,40.650,33.794
4,Anguilla,2021-09-03,62,0,63.278,59.759
...,...,...,...,...,...,...
218,Saba,2021-09-03,1,0,NaN,67.253
219,Yemen,2021-09-03,240,39,1.000,0.045
220,South Africa,2021-09-03,49387,1727,11.100,10.092
221,Zambia,2021-09-03,1001,21,1.653,1.086
